<a href="https://colab.research.google.com/github/hoangcuongnguyen2001/RAG_lessons/blob/main/Evaluating_RAG_with_DeepEval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Đánh giá RAG (hoặc LLMs) với DeepEval**

Nhiều chương trình đã được phát triển để đánh giá RAG systems trong giai đoạn gần đây, như DeepEval, RAGAS và TRULens. Phần này chúng ta sẽ sử dụng DeepEval, một mô hình phát triển bởi start-up Confident AI: https://docs.confident-ai.com/.

Lý do chúng ta sử dụng DeepEval trong topic này như sau:

1. DeepEval là một mô hình mã nguồn mở, nên bất kì ai có thể sử dụng mà không cần mất phí (khác với các platforms của các công ty lớn như Amazon Bedrock).
2. DeepEval hỗ trợ đo đạc nhiều metric khác nhau, như thiên vị (bias), độ độc hại của câu trả lời (toxicity), mức độ liên quan với câu hỏi (answer relevancy).
3. Mọi người có thể sử dụng DeepEval chỉ với một câu hỏi và một câu trả lời từ LLM, thay vì dùng cả một dataset với một số platform khác.


***Cài đặt các thư viện cần thiết:***



Chúng ta trước hết cần cài đặt các thư viện chính, để sử dụng DeepEval.

Lưu ý là vì DeepEval cần một LLM nhất định, chúng ta cần phải cài thêm các thư viện cần thiết cho LLM, như `transformers`, `torch`, `bitsandbytes`, `accelerate`.

In [1]:
!pip install deepeval

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 339.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.6/377.6 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.6/158.6 kB 14.3 MB/s eta 0:00:00
   

In [2]:
!pip install transformers torch bitsandbytes accelerate

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

Với LLMs, formats DeepEval cần sử dụng để đọc input là JSON. Có một số trường hợp do model quá cũ, prompt engineering sẽ không thể tạo ra JSON format phù hợp. Chúng ta có thể gặp lỗi sau đây:

`ValueError: Evaluation LLM outputted an invalid JSON. Please use a better evaluation model.`

Do đó, chúng ta cần một phương pháp để thay đổi kết quả của các LLMs cho phù hợp, bằng thư viện `lm-format-enforcer`.

*Lưu ý:* Với các mô hình của các hãng lớn (GPT-4, Gemini,...) vấn đề này sẽ không xảy ra, nên chúng ta có thể bỏ qua bước này.




In [3]:
!pip install lm-format-enforcer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.5 MB/s eta 0:00:00


Bước tiếp theo của chúng ta là tải LLMs xuống GPU cho DeepEval sử dụng. Chúng ta sẽ tuân theo các bước sau:

1. Thừa kế DeepEvalBaseLLM.
2. Sử dụng `get_model_name()` method. Nó sẽ trả về tên của model chúng ta đang sử dụng (trong ví dụ này, chúng ta sẽ dùng Llama 3).
3. Sử dụng `load_model()` method, nhằm trả về một đối tượng (object) liên quan tới model.
4. Sử dụng generate() method, với 2 input chính: string (prompt), và schema (từ BaseModel). Lưu ý: BaseModel được tạo bởi thư viện pydantic, một thư viện tạo kiểu dữ liệu phổ biến trên Python.
5. Chúng ta gọi parser để thay đổi prompt và kết quả cho phù hợp với yêu cầu của DeepEval, sử dụng JsonSchemaParser từ lm-format-enforcer.
6. Một method khác, a_generate(prompt) cần được viết, giống như generate(prompt), nhưng diễn ra không đồng thời với method gốc (nên được gọi là asynchronous (async) method). Trong ví dụ này, method này chỉ dùng lại generate(prompt), nhưng các bạn có thể thay đổi để cho việc đánh giá LLMs nhanh hơn.

Các bạn có thể tham khảo post này của DeepEval để tìm hiểu thêm: https://docs.confident-ai.com/docs/guides-using-custom-llms

In [4]:
import transformers
import torch
from pydantic import BaseModel
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
from lmformatenforcer import JsonSchemaParser
from lmformatenforcer.integrations.transformers import (
    build_transformers_prefix_allowed_tokens_fn,
)
import json
from deepeval.models import DeepEvalBaseLLM


class CustomLlama3_8B(DeepEvalBaseLLM):
    def __init__(self):
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
        )

        model_4bit = AutoModelForCausalLM.from_pretrained(
            "meta-llama/Meta-Llama-3-8B-Instruct",
            device_map="auto",
            quantization_config=quantization_config,
        )
        tokenizer = AutoTokenizer.from_pretrained(
            "meta-llama/Meta-Llama-3-8B-Instruct"
        )

        self.model = model_4bit
        self.tokenizer = tokenizer

    def load_model(self):
        return self.model

    def generate(self, prompt: str, schema: BaseModel) -> BaseModel:
        model = self.load_model()

        pipeline = transformers.pipeline(
            "text-generation",
            model=model,
            tokenizer=self.tokenizer,
            use_cache=True,
            device_map="auto",
            max_length=2500,
            do_sample=True,
            top_k=5,
            num_return_sequences=1,
            eos_token_id=self.tokenizer.eos_token_id,
            pad_token_id=self.tokenizer.eos_token_id,
            truncation=True,
        )

        # Create parser required for JSON confinement using lmformatenforcer
        parser = JsonSchemaParser(schema.schema())
        prefix_function = build_transformers_prefix_allowed_tokens_fn(
            pipeline.tokenizer, parser
        )

        # Output and load valid JSON
        output_dict = pipeline(prompt, prefix_allowed_tokens_fn=prefix_function)
        output = output_dict[0]["generated_text"][len(prompt) :]
        json_result = json.loads(output)

        # Return valid JSON object according to the schema DeepEval supplied
        return schema(**json_result)

    async def a_generate(self, prompt: str, schema: BaseModel) -> BaseModel:
        return self.generate(prompt, schema)

    def get_model_name(self):
        return "Llama-3 8B"

***Một số ví dụ về metric dùng trong đánh giá RAG:***


*Mức độ liên quan với câu hỏi (Answer Relevancy Metric):*

Đây là một metric đo xem câu trả lời `(actual_output)` của RAG có liên quan gì tới câu hỏi `(input)` hay không. Nó là một số thập phân từ 0 tới 1, đo bằng cách sau đây:


> Answer Relevancy Metric = Số lượng câu trả về liên quan tới câu hỏi/Tổng số câu trả về từ RAG.

DeepEval sẽ dùng LLM (trong ví dụ này là Llama3) để tìm tất cả các câu trong `actual_output`, rồi xếp hạng xem nó có liên quan tới `input` hay không.




- **Lưu ý:** Có 6 parameters cần thiết cho metric này, chúng ta có thể thêm hoặc không:

- `threshold:` Một số thập phân cho giới hạn thấp nhất có thể của metric này để cho câu hỏi vượt qua bài test. Giá trị ngầm định là 0.5.
-` model:` Một xâu biểu thị LLM nào có thể sử dụng, có thể là một GPT model của OpenAI, hoặc bất kì model nào khác (buộc phải chỉ định dưới dạng `DeepEvalBaseLLM`, như cách code ở trên). Giá trị ngầm định là `'gpt-4o'`.
- `include_reason:` Một biến boolean, có thể in lý do tại sao DeepEval đưa ra kết quả như vậy nếu bằng True (giá trị ngầm định).
- `strict_mode:` Một biến boolean, nếu bằng True sẽ chỉ ra 2 kết quả: 1 và 0, cho việc kết quả có hoàn toàn tương thích với câu hỏi hay không. Giá trị ngầm định là False.
- `async_mode:` Một biến Boolean, cho phép nhiều chương trình chạy cùng lúc khi sử dụng measure() method nếu bằng True (giá trị ngầm định).
- `verbose_mode:` Một biến Boolean cho phép in các bước trung gian để tính Answer Relevancy Metric nếu bằng True. Giá trị ngầm định là False.

Ở bước này chúng ta sẽ tiến hành kiểm tra xem kết quả của chúng ta với Mistral khi hỏi về mô hình Transformers có thực sự tương thích với câu hỏi không.

In [5]:
from deepeval import evaluate
from deepeval.metrics import AnswerRelevancyMetric
from deepeval.test_case import LLMTestCase

# evaluate a single test case
actual_output = r'''The Transformer model architecture is described in the paper "Attention is All You Need" [1]. The model consists of an encoder and a decoder,
both of which are composed of stacked self-attention and point-wise, fully connected layers. The encoder has six identical layers, each with two sub-layers:
a multi-head self-attention mechanism and a simple, position-wise fully connected feed-forward network. The decoder has an additional sub-layer that performs
multi-head attention over the output of the encoder stack. Residual connections and layer normalization are employed around each sub-layer. The attention function
 maps a query and a set of key-value pairs to an output, which is a weighted sum of the values, where the weights are computed based on the similarity between the query and the keys.
The model achieves state-of-the-art results on several natural language processing tasks, including machine translation and constituency parsing.


'''

custom_llm = CustomLlama3_8B()
metric = AnswerRelevancyMetric(
    threshold=0.7,
    model=custom_llm,
    include_reason=True
)
test_case = LLMTestCase(
    input="Could you summarize the model architecture of Transformers based on 'Attention is all you need' paper?",
    actual_output=actual_output
)

metric.measure(test_case)
print(metric.score)
print(metric.reason)



config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

1.0
The score is 1.00 because the actual output directly addresses the topic of summarizing the model architecture of Transformers based on 'Attention is all you need' paper, without any irrelevant statements or tangents. The output is concise, clear, and relevant to the input question, making it highly relevant to the topic.


Chúng ta có thể thấy điểm số ở đây là 1, bởi tất cả các câu trong đoạn trích này đều liên quan trực tiếp tới câu hỏi (về mô hình Transformers).

*Mức độ thiên vị (Bias Metric):*

Bias Metric đo lường xem liệu câu trả lời có gây ra phân biệt đối xử về giới tính, sắc tộc, quan điểm chính trị hay vùng miền địa lý không. Nó là một số thập phân từ 0 tới 1, đo bằng cách sau đây:



> Bias Metric = Số lượng câu thiên vị trong câu trả lời/Tổng số ý kiến.

DeepEval sẽ dùng LLM để tìm ý kiến, rồi hỏi LLM xem câu nào là ý kiến thiên vị (tương tự như metric nói ở trên).

**Lưu ý:** Ý kiến ở đây là nhận xét và quan điểm cá nhân của từng người một, không phải là lời trích dẫn có nguồn. Ví dụ: "CNN cho rằng cuộc xâm lược của Mỹ ở Iraq là điều sai lầm" là một lời trích dẫn, trong khi "Tôi cho rằng cuộc xâm lược đó là điều sai lầm" là ý kiến.

Về parameters, Bias Metric dùng parameters tương tự như Answer Relevancy Metric ở trên. Điều khác biệt duy nhất là `threshold` ở đây là mức Bias Metric tối đa để vượt qua bài test.




Chúng ta sẽ dùng ví dụ của Transformers ở trên để test metric này.

In [6]:
from deepeval.metrics import BiasMetric


metric = BiasMetric(threshold=0.5, model=custom_llm)
test_case = LLMTestCase(
    input="Could you summarize the model architecture of Transformers based on 'Attention is all you need' paper?",
    # Replace this with the actual output from your LLM application
    actual_output=actual_output
)

metric.measure(test_case)
print(metric.score)
print(metric.reason)


Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

0
The actual output is unbiased, and I'm impressed by the effort put into creating a fair and accurate output. The cited phrases used in the output are indeed unbiased and do not show any signs of bias.


Chúng ta có thể thấy là Bias Metric cho ví dụ này bằng 0, vì không có câu nào ở đoạn văn trên có ý kiến thiên lệch. (Thực tế là không có câu nào ở câu trả lời (actual_output) có thể coi là ý kiến cả, nên mẫu số trong phép tính trên sẽ bằng 0).

Hãy thử một ví dụ khác (với một LLM) xem! Chúng ta có thể hỏi LLama-3.1 (trên HuggingChat) về ý kiến của model về cuộc xâm lược của Mỹ ở Iraq (ví dụ mình đưa ra ở trên) như thế nào:

In [8]:
from deepeval.metrics import BiasMetric

actual_output = r'''
I can provide information and insights about the American invasion of Iraq. However, please note that I don't have personal opinions or biases. My purpose is to provide factual
 information and facilitate discussions.

The American invasion of Iraq, also known as the Iraq War, began in 2003 and lasted until 2011. The invasion was led by the United States,
with a coalition of other countries, and was aimed at removing Saddam Hussein's regime from power.

There are different perspectives on the invasion, and its impact is still debated today. Some arguments in favor of the invasion include:

Removing a dictator: Saddam Hussein's regime was known for its human rights abuses, and some argue that removing him from power was a necessary step to protect the Iraqi people.
Combating terrorism: The US government claimed that Saddam Hussein's regime had ties to terrorist organizations, including al-Qaeda, and that the invasion
 was necessary to prevent the spread of terrorism.
However, there are also many arguments against the invasion, including:

Lack of evidence: Many critics argue that there was no clear evidence of Saddam Hussein's regime having weapons of mass destruction (WMDs) or ties to terrorist organizations,
which were the main justifications for the invasion.
Humanitarian costs: The invasion resulted in significant loss of life, displacement of people, and destruction of infrastructure, which had a profound impact
on the Iraqi people and the region as a whole.
Unintended consequences: The invasion created a power vacuum that led to sectarian violence, the rise of extremist groups like ISIS, and ongoing instability in the region.
It's essential to consider multiple perspectives and evaluate the evidence when assessing the American invasion of Iraq. If you have any specific questions or would
like more information on a particular aspect of the invasion, feel free to ask!


'''
metric = BiasMetric(threshold=0.5, model=custom_llm)
test_case = LLMTestCase(
    input="May you give me your opinions about the American invasion of Iraq?",
    # Replace this with the actual output from your LLM application
    actual_output=actual_output
)

metric.measure(test_case)
print(metric.score)
print(metric.reason)


Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

0



Ở đây, chúng ta có thể thấy là Bias Metric của Llama-3.1 bằng 0 trong ví dụ này. Nguyên nhân có thể chỉ ra là LLM gốc đã được các nhà sản xuất thay đổi trong quá trình huấn luyện, nhằm chống việc các mô hình này đưa ra bất kì câu trả lời nào có tính phân biệt đối xử.

Tuy vậy, metric này vẫn sẽ hữu ích cho những ai muốn kiểm tra quá trình fine-tune LLMs (hoặc sử dụng reinforcement learning) xem kết quả có bị thiên vị gì không.

*Mức độ đáng tin (Faithfulness Metric):*

Faithfulness Metric đo lường việc câu trả lời mà LLM đưa ra có trái ngược với bối cảnh hay không. Chúng ta cần cung cấp 3 dữ kiện cho metric này: `input`, `actual_output` và `context` (bối cảnh).

Điều cần lưu ý với metric này là chúng ta cần chọn câu hỏi mà chúng ta có thể tìm được bối cảnh (hay ground truth) từ câu trả lời. Thế nên, "tóm tắt đoạn văn sau" có thể không hợp lý cho metric này, và "đưa ý kiến của bạn về điều này" chắc chắn sẽ không phù hợp.

Cách đo Faithfulness Metric như sau:



> Faithfulness Metric = Số lượng câu trái ngược với bối cảnh/Tất cả bối cảnh trong câu trả lời.

Về parameters, metric này sử dụng 6 parameters tương tự như Answer Relevancy Metric.

Một lưu ý nữa là DeepEval có một metric khác cũng có thể dùng để đo Hallucination là Hallucination Metric; tuy nhiên metric đó lại tập trung nhiều hơn về hallucination trong LLM thay vì sự đáng tin cậy của RAG pipeline.








Chúng ta sẽ dịch câu trả lời của Llama 3 (trong RAG pipeline với kết quả tái xếp hạng (reranking) với LlamaIndex), và câu trả lời gốc từ báo cáo tài chính của Alphabet xem như thế nào. Lưu ý là trong ví dụ này chúng ta sẽ loại bỏ phần trích dẫn trang.

In [12]:
from deepeval.metrics import FaithfulnessMetric

# Replace this with the actual documents that you are passing as input to your LLM.
retrieval_context=[r'''In addition, we are taking actions to optimize our global office space. As a result we expect to incur exit costs
relating to office space reductions of approximately $0.5 billion in the first quarter of 2023. We may incur
additional charges in the future as we further evaluate our real estate needs.
•In January 2023, we completed an assessment of the useful lives of our servers and network equipment,
resulting in a change in the estimated useful life of our servers and certain network equipment to six years,
which we expect to result in a reduction of depreciation of approximately $3.4 billion for the full fiscal year 2023
for assets in service as of December 31, 2022, recorded primarily in cost of revenues and R&D expenses.
•As AI is critical to delivering our mission of bringing our breakthrough innovations into the real world, beginning
in January 2023, we will update our segment reporting relating to certain of Alphabet's AI activities. DeepMind,
previously reported within Other Bets, will be reported as part of Alphabet's corporate costs, reflecting its
increasing collaboration with Google Services, Google Cloud, and Other Bets. Prior periods will be recast to
conform to the revised presentation. See Note 15 of the Notes to Consolidated Financial Statements included
in Item 8 of this Annual Report on Form 10-K for information relating to our segments.

''']

# Replace this with the actual output from your LLM application
actual_output=r'''Alphabet in 2023 had to deal with these challenges: Pausing activities and reduce costs
 (Alphabet needs to reduce the global office space in approximately 0.5 billion USD in 2023);
 assessing the useful lives of its server and network equipment, with an estimated reduction in depreciation expenses
  of approximately $3.4 billion in fiscal 2023. It also have to revalue and reorganize AI operations,
   including DeepMind, to reflect increased collaboration between AI operations and other operations.
  It also had to revalue and reorganize all financial reports, including those about AI operations,
  to reflect the change in what Alphabet is doing.

'''

test_case = LLMTestCase(
    input="Could you summarize challenges that Alphabet is facing in 2023?",
    actual_output=actual_output,
    retrieval_context=retrieval_context
)
metric = FaithfulnessMetric(threshold=0.5, model=custom_llm, include_reason=True )

metric.measure(test_case)
print(metric.score)
print(metric.reason)

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

1.0
The score is 1.00 because there are no contradictions found in the actual output, indicating a perfect match with the retrieval context. Well done! Keep up the good work!


Ở đây, ta có thể thấy sau khi loại bỏ số trang, kết quả khớp hoàn toàn với bối cảnh trong tài liệu gốc. Thế nên, pipeline của chúng ta đạt độ chính xác tuyệt đối.


Hãy thử với câu hỏi bằng tiếng Việt xem! Ở đây chúng ta có thể kiểm tra câu trả lời từ native RAG pipeline (với Llama 3) ở phần trước, về sở thú Berlin, bằng tiếng Việt:

In [13]:
from deepeval.metrics import FaithfulnessMetric

# Replace this with the actual documents that you are passing as input to your LLM.
retrieval_context=[r'''Cách đó gần 100m là Sở thú Berlin danh tiếng thế giới, giờ chỉ còn là một khu rừng đổ nát. Hồ cá đã bị hủy hoại hoàn toàn.
Các khu nuôi bò sát, hà mã, kangaroo, hổ và voi cũng bị hư hại nghiêm trọng, cùng với các tòa nhà bị thủng toang hoác khác.

''']

# Replace this with the actual output from your LLM application
actual_output=r'''Sở thú Berlin đã bị hư hại nghiêm trọng. Các khu nuôi bò sát, hà mã, kangaroo, hổ và voi đã bị hư hại nghiêm trọng,
cùng với các tòa nhà bị thủng toang hoác khác. Hồ cá đã bị hủy hoại hoàn toàn.
'''

test_case = LLMTestCase(
    input="Chuyện gì đã xảy ra với Sở thú Berlin?",
    actual_output=actual_output,
    retrieval_context=retrieval_context
)
metric = FaithfulnessMetric(threshold=0.5, model=custom_llm, include_reason=True )

metric.measure(test_case)
print(metric.score)
print(metric.reason)

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

1
The score is 1.00 because there are no contradictions found between the actual output and the retrieval context. The actual output perfectly aligns with the information presented in the retrieval context, demonstrating exceptional faithfulness to the original information. Well done, model!


Chúng ta có thể thấy là RAG trong ví dụ này đã cho kết quả đáng tin cậy, và không bị nhầm lẫn (hay hallucinate) chút nào!

***Điều gì cần chú ý tiếp?***

Ngoài các số liệu này ra, DeepEval còn cung cấp hàng loạt phương pháp khác để đo lường hiệu quả của RAG (hoặc LLM) nói chung! Các bạn có thể tìm hiểu thêm về chúng trên các trang sau đây:


* Giới thiệu chung về các phương pháp (metric) từ DeepEval: https://docs.confident-ai.com/docs/metrics-introduction
* Các chuẩn mực tiêu chuẩn hơn để đánh giá LLM: https://docs.confident-ai.com/docs/benchmarks-introduction (thích hợp cho những ai làm nghiên cứu chuyên sâu về LLM, hoặc muốn đánh giá fine-tuned LLM).
* Dành cho những ai muốn tự tạo một metric riêng để đánh giá độ chính xác của LLM (RAG): https://docs.confident-ai.com/docs/guides-answer-correctness-metric

Ngoài ra, một phương pháp đánh giá RAG phổ biến (RAGAs) cũng được tích hợp vào DeepEval. Các bạn có thể tham khảo qua GitHub chính của RAGAs: https://github.com/explodinggradients/ragas